# Visualização de NDVI e Previsões com ARIMA em Mapas Interativos

Este notebook carrega imagens NDVI, treina um modelo ARIMA para fazer previsões de séries temporais,
e exibe as imagens e previsões em mapas interativos usando a biblioteca **Folium**.

## Passos:
1. Carregar dados NDVI das imagens .tif.
2. Treinar um modelo ARIMA para prever os NDVIs futuros.
3. Exibir os NDVIs e as previsões em um mapa interativo.



In [6]:

import os
import numpy as np
import rasterio
import folium
from rasterio.plot import show
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from folium import plugins


In [7]:
def create_ndvi_map(image_file, map_center, zoom_start=10):
    with rasterio.open(image_file) as src:
        ndvi_image = src.read(1)
        ndvi_bounds = src.bounds
        ndvi_min, ndvi_max = np.nanmin(ndvi_image), np.nanmax(ndvi_image)

        # Criar o mapa centrado nas coordenadas fornecidas
        ndvi_map = folium.Map(location=map_center, zoom_start=zoom_start)

        # Converter NDVI para HTMLOverlay, que será sobreposto ao mapa
        plt.figure(figsize=(8, 6))

        # Mostrar a imagem com imshow para que possamos associar a colorbar
        img = plt.imshow(ndvi_image, cmap='RdYlGn', vmin=ndvi_min, vmax=ndvi_max)

        # Adicionar a colorbar associada à imagem plotada
        plt.colorbar(img, label='NDVI')

        plt.title('Imagem NDVI')
        plt.savefig('ndvi_image.png')
        plt.close()

        # Adicionar imagem ao mapa usando Overlay de imagem
        ndvi_overlay = folium.raster_layers.ImageOverlay(
            image='ndvi_image.png',
            bounds=[[ndvi_bounds.bottom, ndvi_bounds.left],
                    [ndvi_bounds.top, ndvi_bounds.right]],
            opacity=0.6
        )
        ndvi_overlay.add_to(ndvi_map)

        return ndvi_map


In [8]:

def load_and_sample_data(image_folder, sample_size=1000):
    ndvi_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".tif")])
    ndvi_timeseries = []

    for file in ndvi_files:
        file_path = os.path.join(image_folder, file)
        with rasterio.open(file_path) as src:
            ndvi_image = src.read(1)
            flat_image = ndvi_image.flatten()
            sampled_pixels = np.random.choice(flat_image, size=sample_size, replace=False)
            ndvi_timeseries.append(np.mean(sampled_pixels))

    return np.array(ndvi_timeseries)


In [9]:

def train_arima_model(data):
    model = ARIMA(data, order=(1, 1, 1))
    results = model.fit()
    return results

def make_predictions(model, n_periods=5):
    forecast = model.forecast(steps=n_periods)
    return forecast


In [10]:

def add_prediction_layer(ndvi_map, predictions, map_center):
    # Simular uma visualização de previsão NDVI para a camada
    folium.Marker(location=map_center, popup=f"Previsão NDVI: {predictions[0]:.2f}").add_to(ndvi_map)


In [11]:

def display_ndvi_maps(image_folder, map_center, zoom_start=10):
    # Carregar e normalizar os dados
    print("Carregando e amostrando dados...")
    ndvi_data = load_and_sample_data(image_folder)
    scaler = MinMaxScaler(feature_range=(0, 1))
    ndvi_scaled = scaler.fit_transform(ndvi_data.reshape(-1, 1)).flatten()

    # Treinar modelo ARIMA
    print("Treinando modelo ARIMA...")
    model = train_arima_model(ndvi_scaled)

    # Fazer previsões
    print("Fazendo previsões...")
    predictions_scaled = make_predictions(model, n_periods=5)
    predictions = scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()

    # Criar o mapa com a primeira imagem NDVI
    ndvi_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".tif")])
    ndvi_map = create_ndvi_map(os.path.join(image_folder, ndvi_files[0]), map_center, zoom_start)

    # Adicionar camada de previsão ao mapa
    add_prediction_layer(ndvi_map, predictions, map_center)

    # Adicionar controle de camadas
    folium.LayerControl().add_to(ndvi_map)

    # Exibir mapa
    return ndvi_map


In [12]:

def main(image_folder, output_folder, map_center):
    # Criar o mapa interativo com os NDVIs e previsões
    ndvi_map = display_ndvi_maps(image_folder, map_center)

    # Salvar o mapa interativo como arquivo HTML
    output_file = os.path.join(output_folder, 'ndvi_map.html')
    ndvi_map.save(output_file)
    print(f"Mapa interativo salvo em {output_file}")


In [13]:

# Exemplo de uso
image_folder = "./Angola_MODIS_Images"  # Pasta com as imagens NDVI
output_folder = "./resultados"  # Pasta para salvar resultados
map_center = [-12.5, 17.5]  # Coordenadas do centro do mapa (latitude, longitude)

main(image_folder, output_folder, map_center)


Carregando e amostrando dados...
Treinando modelo ARIMA...
Fazendo previsões...
Mapa interativo salvo em ./resultados/ndvi_map.html
